In [34]:
import re
import requests
import json
import pandas as pd

BASE_URL = 'https://www.zillow.com/async-create-search-page-state'
HEADERS = {
    'pragma': 'no-cache',
    'cache-control': 'no-cache',
    'dnt': '1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
}

payloads = [
    '''
    {"searchQueryState":{"pagination":{},"isMapVisible":true,"mapBounds":{"west":-141.191414625,"east":-97.421883375,"south":35.04431931445853,"north":39.72064307616386},"mapZoom":5,"regionSelection":[{"regionId":9,"regionType":2}],"filterState":{"sortSelection":{"value":"globalrelevanceex"},"isForSaleByAgent":{"value":false},"isNewConstruction":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"isForSaleForeclosure":{"value":false}},"category":"cat2","isListVisible":true},"wants":{"cat2":["mapResults"]},"requestId":2,"isDebugRequest":false}
    ''',
    '''
    {"searchQueryState":{"pagination":{},"isMapVisible":true,"mapBounds":{"west":-141.191414625,"east":-97.42188337500001,"south":32.65410422977953,"north":41.8989428391672},"mapZoom":5,"regionSelection":[{"regionId":9,"regionType":2}],"filterState":{"sortSelection":{"value":"globalrelevanceex"},"isForSaleByAgent":{"value":false},"isNewConstruction":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"isForSaleForeclosure":{"value":false}},"category":"cat2","isListVisible":true},"wants":{"cat2":["mapResults"]},"requestId":3,"isDebugRequest":false}
    '''
]

responses = []

In [35]:
for payload in payloads:    #Populating results list from all requests with variant payloads
    r = requests.put(url=BASE_URL, json=json.loads(payload), headers=HEADERS)
    assert r.status_code == 200, f'Request failed. status code: {r.status_code}'
    
    try: 
        print("Found items:", len(r.json()['cat2']['searchResults']['mapResults']))
        responses.extend(r.json()['cat2']['searchResults']['mapResults'])
    except KeyError:
        print("Non-existing key. Available keys are:\n", r.json().keys())    
print(len(responses))

Found items: 268
Found items: 500
768


In [36]:
df = pd.DataFrame(responses)
df

,zpid,rawHomeStatusCd,marketingStatusSimplifiedCd,imgSrc,hasImage,detailUrl,statusType,statusText,price,priceLabel,...,minArea,variableData,buildingId,isBuilding,canSaveBuilding,unitCount,streetViewURL,streetViewMetadataURL,lotAreaString,lotId
0,17656163,ForSale,For Sale by Owner,https://photos.zillowstatic.com/fp/7efc55bbf0c...,True,/homedetails/1114-Melrose-Ave-Roseville-CA-956...,FOR_SALE,For sale by owner,"$525,000",$525K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,https://photos.zillowstatic.com/fp/39fcaeddc48...,True,"/b/building/36.82021,-119.69247_ll/",FOR_SALE,For Rent,"From $495,000",NaN,...,1863.0,{},36.82021--119.69247,True,False,2.0,NaN,NaN,NaN,NaN
2,68989682,ForSale,For Sale by Owner,https://photos.zillowstatic.com/fp/d046d10411a...,True,/homedetails/1747-N-Hughes-Ave-Clovis-CA-93619...,FOR_SALE,For sale by owner,"$609,000",$609K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2053090597,ForSale,For Sale by Owner,https://maps.googleapis.com/maps/api/staticmap...,NaN,/homedetails/200-S-Dolliver-St-SPACE-292-Pismo...,FOR_SALE,For sale by owner,"$109,950",$110K,...,NaN,NaN,NaN,NaN,NaN,NaN,https://maps.googleapis.com/maps/api/streetvie...,https://maps.googleapis.com/maps/api/streetvie...,NaN,NaN
4,2053121433,ForSale,For Sale by Owner,https://photos.zillowstatic.com/fp/5d7489930d5...,True,/homedetails/114-San-Fernando-Ave-Morro-Bay-CA...,FOR_SALE,For sale by owner,"$490,000",$490K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,2065529961,ForSale,For Sale by Owner,https://maps.googleapis.com/maps/api/staticmap...,NaN,/homedetails/5149-Hess-Rd-Phelan-CA-92371/2065...,FOR_SALE,For sale by owner,"$439,000",$439K,...,NaN,NaN,NaN,NaN,NaN,NaN,https://maps.googleapis.com/maps/api/streetvie...,https://maps.googleapis.com/maps/api/streetvie...,20 acres,NaN
764,95444823,ForSale,For Sale by Owner,https://photos.zillowstatic.com/fp/840bef66f4e...,True,/homedetails/4965-Beckwourth-Ct-Oroville-CA-95...,FOR_SALE,For sale by owner,"$139,900",$140K,...,NaN,"{'type': 'PRICE_REDUCTION', 'text': '$10,000 (...",NaN,NaN,NaN,NaN,NaN,NaN,0.28 acres,NaN
765,2056834923,ForSale,For Sale by Owner,https://photos.zillowstatic.com/fp/861354edeea...,True,/homedetails/93-Juniper-Rd-Chilcoot-CA-96105/2...,FOR_SALE,For sale by owner,"$396,500",$397K,...,NaN,"{'type': 'PRICE_REDUCTION', 'text': '$43,500 (...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
766,112418335,ForSale,For Sale by Owner,https://photos.zillowstatic.com/fp/0fffabd7a86...,True,/homedetails/4622-Kaweah-Rd-Kelseyville-CA-954...,FOR_SALE,For sale by owner,"$13,888",$14K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"8,712 sqft",NaN


In [6]:
#pd.set_option('display.max_columns', None)

df.iloc[0:2]

In [39]:
responses[10]

{'zpid': '2057660575',
 'rawHomeStatusCd': 'ForSale',
 'marketingStatusSimplifiedCd': 'For Sale by Owner',
 'imgSrc': 'https://photos.zillowstatic.com/fp/5118f8eaa8a42c27cffd1951a79f2d17-p_e.jpg',
 'hasImage': True,
 'detailUrl': '/homedetails/200-S-Dolliver-St-SPACE-162-Pismo-Beach-CA-93449/2057660575_zpid/',
 'statusType': 'FOR_SALE',
 'statusText': 'For sale by owner',
 'price': '$235,000',
 'priceLabel': '$235K',
 'address': '200 S Dolliver St SPACE 162, Pismo Beach, CA 93449',
 'beds': 1,
 'baths': 1.0,
 'area': 399,
 'latLong': {'latitude': 35.13428, 'longitude': -120.63384},
 'variableData': {'type': 'TIME_ON_INFO',
  'text': '21 hours ago',
  'data': {'isRead': None, 'isFresh': False}},
 'hdpData': {'homeInfo': {'zpid': 2057660575,
   'streetAddress': '200 S Dolliver St SPACE 162',
   'zipcode': '93449',
   'city': 'Pismo Beach',
   'state': 'CA',
   'latitude': 35.13428,
   'longitude': -120.63384,
   'price': 235000.0,
   'datePriceChanged': 1706169600000,
   'bathrooms': 1.0

In [40]:
to_keep = ['address', 'price', 'beds', 'baths', 'area', 'detailUrl']
df = df[to_keep]
df

,address,price,beds,baths,area,detailUrl
0,"1114 Melrose Ave, Roseville, CA 95678","$525,000",3.0,2.0,1328.0,/homedetails/1114-Melrose-Ave-Roseville-CA-956...
1,"327 McKelvy Ave, Clovis, CA","From $495,000",NaN,NaN,NaN,"/b/building/36.82021,-119.69247_ll/"
2,"1747 N Hughes Ave, Clovis, CA 93619","$609,000",4.0,3.0,2721.0,/homedetails/1747-N-Hughes-Ave-Clovis-CA-93619...
3,"200 S Dolliver St SPACE 292, Pismo Beach, CA 9...","$109,950",1.0,1.0,600.0,/homedetails/200-S-Dolliver-St-SPACE-292-Pismo...
4,"114 San Fernando Ave, Morro Bay, CA 93442","$490,000",2.0,2.0,968.0,/homedetails/114-San-Fernando-Ave-Morro-Bay-CA...
...,...,...,...,...,...,...
763,"5149 Hess Rd, Phelan, CA 92371","$439,000",NaN,0.0,NaN,/homedetails/5149-Hess-Rd-Phelan-CA-92371/2065...
764,"4965 Beckwourth Ct, Oroville, CA 95966","$139,900",NaN,0.0,NaN,/homedetails/4965-Beckwourth-Ct-Oroville-CA-95...
765,"93 Juniper Rd, Chilcoot, CA 96105","$396,500",1.0,1.0,1316.0,/homedetails/93-Juniper-Rd-Chilcoot-CA-96105/2...
766,"4622 Kaweah Rd, Kelseyville, CA 95451","$13,888",NaN,0.0,NaN,/homedetails/4622-Kaweah-Rd-Kelseyville-CA-954...


In [37]:
help(df.drop_duplicates)

Help on method drop_duplicates in module pandas.core.frame:

drop_duplicates(subset: 'Hashable | Sequence[Hashable] | None' = None, *, keep: "Literal['first', 'last', False]" = 'first', inplace: 'bool' = False, ignore_index: 'bool' = False) -> 'DataFrame | None' method of pandas.core.frame.DataFrame instance
    Return DataFrame with duplicate rows removed.
    
    Considering certain columns is optional. Indexes, including time indexes
    are ignored.
    
    Parameters
    ----------
    subset : column label or sequence of labels, optional
        Only consider certain columns for identifying duplicates, by
        default use all of the columns.
    keep : {'first', 'last', False}, default 'first'
        Determines which duplicates (if any) to keep.
        - ``first`` : Drop duplicates except for the first occurrence.
        - ``last`` : Drop duplicates except for the last occurrence.
        - False : Drop all duplicates.
    inplace : bool, default False
        Whether to 

In [83]:
df = df.drop_duplicates()
df.shape

(556, 6)

In [21]:
help(pd.DataFrame.apply)

Help on function apply in module pandas.core.frame:

apply(self, func: 'AggFuncType', axis: 'Axis' = 0, raw: 'bool' = False, result_type: "Literal['expand', 'reduce', 'broadcast'] | None" = None, args=(), **kwargs)
    Apply a function along an axis of the DataFrame.
    
    Objects passed to the function are Series objects whose index is
    either the DataFrame's index (``axis=0``) or the DataFrame's columns
    (``axis=1``). By default (``result_type=None``), the final return type
    is inferred from the return type of the applied function. Otherwise,
    it depends on the `result_type` argument.
    
    Parameters
    ----------
    func : function
        Function to apply to each column or row.
    axis : {0 or 'index', 1 or 'columns'}, default 0
        Axis along which the function is applied:
    
        * 0 or 'index': apply function to each column.
        * 1 or 'columns': apply function to each row.
    
    raw : bool, default False
        Determines if row or column

In [44]:
df.loc[:, 'price'] = df.loc[:, 'price'].apply(lambda row: re.sub(r'[^\d]', '', row))
df

C:\Users\Mido Hany\AppData\Local\Temp\ipykernel_2156\1866998192.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'price'] = df.loc[:, 'price'].apply(lambda row: re.sub(r'[^\d]', '', row))


,address,price,beds,baths,area,detailUrl
0,"1114 Melrose Ave, Roseville, CA 95678",525000,3.0,2.0,1328.0,/homedetails/1114-Melrose-Ave-Roseville-CA-956...
1,"327 McKelvy Ave, Clovis, CA",495000,NaN,NaN,NaN,"/b/building/36.82021,-119.69247_ll/"
2,"1747 N Hughes Ave, Clovis, CA 93619",609000,4.0,3.0,2721.0,/homedetails/1747-N-Hughes-Ave-Clovis-CA-93619...
3,"200 S Dolliver St SPACE 292, Pismo Beach, CA 9...",109950,1.0,1.0,600.0,/homedetails/200-S-Dolliver-St-SPACE-292-Pismo...
4,"114 San Fernando Ave, Morro Bay, CA 93442",490000,2.0,2.0,968.0,/homedetails/114-San-Fernando-Ave-Morro-Bay-CA...
...,...,...,...,...,...,...
757,"4671 Morales St, New Cuyama, CA 93254",225000,3.0,1.0,988.0,/homedetails/4671-Morales-St-New-Cuyama-CA-932...
761,"10675 Bryant St SPACE 2, Yucaipa, CA 92399",89999,1.0,1.0,900.0,/homedetails/10675-Bryant-St-SPACE-2-Yucaipa-C...
763,"5149 Hess Rd, Phelan, CA 92371",439000,NaN,0.0,NaN,/homedetails/5149-Hess-Rd-Phelan-CA-92371/2065...
765,"93 Juniper Rd, Chilcoot, CA 96105",396500,1.0,1.0,1316.0,/homedetails/93-Juniper-Rd-Chilcoot-CA-96105/2...


In [66]:
df['link'] = df['detailUrl'].apply(lambda x: f"https://www.zillow.com/{x}")
df.loc[100, :]

C:\Users\Mido Hany\AppData\Local\Temp\ipykernel_2156\728619470.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['link'] = df['detailUrl'].apply(lambda x: f"https://www.zillow.com/{x}")


address             10479 Boren Bega Dr, Kelseyville, CA 95451
price                                                    12888
beds                                                       NaN
baths                                                      NaN
area                                                       NaN
detailUrl    /homedetails/10479-Boren-Bega-Dr-Kelseyville-C...
link         https://www.zillow.com//homedetails/10479-Bore...
Name: 100, dtype: object

In [70]:
df = df.drop(columns='detailUrl')
df

,address,price,beds,baths,area,link
0,"1114 Melrose Ave, Roseville, CA 95678",525000,3.0,2.0,1328.0,https://www.zillow.com//homedetails/1114-Melro...
1,"327 McKelvy Ave, Clovis, CA",495000,NaN,NaN,NaN,"https://www.zillow.com//b/building/36.82021,-1..."
2,"1747 N Hughes Ave, Clovis, CA 93619",609000,4.0,3.0,2721.0,https://www.zillow.com//homedetails/1747-N-Hug...
3,"200 S Dolliver St SPACE 292, Pismo Beach, CA 9...",109950,1.0,1.0,600.0,https://www.zillow.com//homedetails/200-S-Doll...
4,"114 San Fernando Ave, Morro Bay, CA 93442",490000,2.0,2.0,968.0,https://www.zillow.com//homedetails/114-San-Fe...
...,...,...,...,...,...,...
757,"4671 Morales St, New Cuyama, CA 93254",225000,3.0,1.0,988.0,https://www.zillow.com//homedetails/4671-Moral...
761,"10675 Bryant St SPACE 2, Yucaipa, CA 92399",89999,1.0,1.0,900.0,https://www.zillow.com//homedetails/10675-Brya...
763,"5149 Hess Rd, Phelan, CA 92371",439000,NaN,0.0,NaN,https://www.zillow.com//homedetails/5149-Hess-...
765,"93 Juniper Rd, Chilcoot, CA 96105",396500,1.0,1.0,1316.0,https://www.zillow.com//homedetails/93-Juniper...


In [73]:
df['price'] = df['price'].astype(int)

df.dtypes

In [81]:
df = df.fillna('-')

In [82]:
df

,address,price,beds,baths,area,link
0,"1114 Melrose Ave, Roseville, CA 95678",525000,3.0,2.0,1328.0,https://www.zillow.com//homedetails/1114-Melro...
1,"327 McKelvy Ave, Clovis, CA",495000,-,-,-,"https://www.zillow.com//b/building/36.82021,-1..."
2,"1747 N Hughes Ave, Clovis, CA 93619",609000,4.0,3.0,2721.0,https://www.zillow.com//homedetails/1747-N-Hug...
3,"200 S Dolliver St SPACE 292, Pismo Beach, CA 9...",109950,1.0,1.0,600.0,https://www.zillow.com//homedetails/200-S-Doll...
4,"114 San Fernando Ave, Morro Bay, CA 93442",490000,2.0,2.0,968.0,https://www.zillow.com//homedetails/114-San-Fe...
...,...,...,...,...,...,...
757,"4671 Morales St, New Cuyama, CA 93254",225000,3.0,1.0,988.0,https://www.zillow.com//homedetails/4671-Moral...
761,"10675 Bryant St SPACE 2, Yucaipa, CA 92399",89999,1.0,1.0,900.0,https://www.zillow.com//homedetails/10675-Brya...
763,"5149 Hess Rd, Phelan, CA 92371",439000,-,0.0,-,https://www.zillow.com//homedetails/5149-Hess-...
765,"93 Juniper Rd, Chilcoot, CA 96105",396500,1.0,1.0,1316.0,https://www.zillow.com//homedetails/93-Juniper...


In [86]:
df.to_csv('final_homes.csv', index=False)